RCNN


In [46]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Lambda
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Convolution1D,MaxPooling1D, Flatten
from tensorflow.keras.datasets import imdb
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from tensorflow.keras import callbacks
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
import matplotlib.pyplot as plt 
import matplotlib
matplotlib.use('TkAgg')
import numpy as np
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

In [47]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import tensorflow.keras as keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Lambda
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Convolution1D,MaxPooling1D, Flatten
from tensorflow.keras.datasets import imdb
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import Normalizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from tensorflow.keras import callbacks
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
#
#from PIL import ImageTk, Image
import os

In [48]:
def analysis():
    import tkinter
    
    train_data= pd.read_csv("UNSW_NB15_training-set.csv")
    col_names = ["id"
                 ,"dur","proto"
                 ,"service","state","spkts","dpkts","sbytes","dbytes","rate","sttl","dttl","sload","dload","sloss"
                 ,"dloss","sinpkt","dinpkt","sjit","djit","swin","stcpb","dtcpb","dwin","tcprtt","synack"
                 ,"ackdat","smean","dmean","trans_depth","response_body_len","ct_srv_src","ct_state_ttl","ct_dst_ltm","ct_src_dport_ltm",
                 "ct_dst_sport_ltm","ct_dst_src_ltm","is_ftp_login","ct_ftp_cmd","ct_flw_http_mthd","ct_src_ltm","ct_srv_dst","is_sm_ips_ports","attack_cat","label"]
    test_data = pd.read_csv('UNSW_NB15_testing-set.csv',header=None, names = col_names)
    #represent data in form of bar graph
    
     #distribution of attacks and normal traffic
    f, axes = plt.subplots(2, 2, figsize=(12, 12))
    f.suptitle("---------------------------------\nTrain data length 82332\n---------------------------------\nTest data length 175341\n---------------------------------\n")
    # Create the plots using seaborn package
    sns.countplot(x="label", data=train_data, ax=axes[0,0])
    sns.countplot(x="label", data=test_data, ax=axes[0,1])
    sns.countplot(x="attack_cat", data=train_data, ax=axes[1,0], order = train_data['attack_cat'].value_counts().index)
    sns.countplot(x="attack_cat", data=test_data, ax=axes[1,1], order = test_data['attack_cat'].value_counts().index)

    # plot titles
    axes[0,0].set_title("Training data distribution")
    axes[1,0].set_title("Training data distribution")
    axes[0,1].set_title("Testing data distribution")
    axes[1,1].set_title("Testing data distribution")

    # Rotate xticks for readability
    axes[1,0].tick_params('x', labelrotation=45)
    axes[1,1].tick_params('x', labelrotation=45)

    # Change the xtick labels for attack / normal
    axes[0,0].set_xticklabels(["Normal", "Attack"])
    axes[0,1].set_xticklabels(["Normal", "Attack"])

    # Adding some space between the plots for y labels
    plt.subplots_adjust(wspace=0.5)
    
    
   


In [49]:
def rrcnn():
    
    traindata = pd.read_csv('ddtrain.csv', header=None)
    testdata = pd.read_csv('ddtest.csv', header=None)
    X = traindata.iloc[:,1:42]
    Y = traindata.iloc[:,0]
    C = testdata.iloc[:,0]
    T = testdata.iloc[:,1:42]
    scaler = Normalizer().fit(X)
    trainX = scaler.transform(X)

    scaler = Normalizer().fit(T)
    testT = scaler.transform(T)
    y_train = np.array(Y)
    y_test = np.array(C)
    X_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
    X_test = np.reshape(testT, (testT.shape[0],testT.shape[1],1))
    arr=np.random.randn(0)
    with h5py.File('checkpoint-{epoch:02d}.hdf5','w') as f:
        dset=f.create_dataset("default",data=arr)

    lstm_output_size = 128

    cnn = Sequential()
    cnn.add(Convolution1D(64, 3, padding="same",activation="relu",input_shape=(41, 1)))
    cnn.add(MaxPooling1D(pool_size=(2)))
    cnn.add(Flatten())
    cnn.add(Dense(128, activation="relu"))
    cnn.add(Dropout(0.5))
    cnn.add(Dense(1, activation="sigmoid"))
    print(cnn.summary())
    cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
    # train
    checkpointer = callbacks.ModelCheckpoint(filepath="checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
    csv_logger = CSVLogger('cnntrainanalysis1.csv',separator=',', append=False)
    call=[checkpointer,csv_logger]
    cnn.fit(X_train, y_train,validation_data=(X_test, y_test),callbacks=call)
    #cnn.save("results/cnn1results/cnn_model.hdf5")
    testdata = pd.read_csv('ddtest.csv', header=None)
    C = testdata.iloc[:,0]
    T = testdata.iloc[:,1:42]
    scaler = Normalizer().fit(T)
    testT = scaler.transform(T)
    y_test = np.array(C)
    X_test = np.reshape(testT, (testT.shape[0],testT.shape[1],1))
    arr=np.random.randn(0)
    with h5py.File('checkpoint-{epoch:02d}.hdf5','w') as f:
        dset=f.create_dataset("default",data=arr)

    lstm_output_size = 128

    cnn = Sequential()
    cnn.add(Convolution1D(64, 3, padding="same",activation="relu",input_shape=(41, 1)))
    cnn.add(MaxPooling1D(pool_size=(2)))
    cnn.add(Flatten())
    cnn.add(Dense(128, activation="relu"))
    cnn.add(Dropout(0.5))
    cnn.add(Dense(1, activation="sigmoid"))

    cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
    # train
    checkpointer = callbacks.ModelCheckpoint(filepath="checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
    csv_logger = CSVLogger('cnntrainanalysis1.csv',separator=',', append=False)
    cnn.fit(X_train, y_train,validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
    #cnn.save("results/cnn1results/cnn_model.hdf5")
    cnn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    loss, accuracy = cnn.evaluate(X_test, y_test)
    print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
    y = cnn.predict(X_test)

    #np.savetxt("cnn.txt", y_pred)
    a = accuracy_score(y_test, y.round(),)
    r = recall_score(y_test, y.round()  )
    p = precision_score(y_test, y.round())
    #f1 = f1_score(y_test, y.round())

    label1 = Label(root,bg='white', text='Accuracy Metrics of RCNN:')
    label1.pack()
    label = Label(root,bg='white', text='=============================================')
    label.pack()
    l=[]
    label2 = Label(root, bg='white',text='')
    label3 = Label(root, bg='white',text='')
    label4 = Label(root, bg='white',text='')
    label5 = Label(root, bg='white',text='')
    label2.config(text='Accuracy: '+str(a))
    label2.pack()
    label3.config(text='Recall: '+str(r))
    label3.pack()
    label4.config(text='Precision:'+str(p))
    label4.pack()
    label5 = Label(root,bg='white', text='=============================================')
    label5.pack()
    l.append(a)
    l.append(r)
    l.append(p)
    return l

naive bayes

In [50]:
def NB(file):
    import pandas as pd
    col_names = ["id"
                 ,"dur","proto"
                 ,"service","state","spkts","dpkts","sbytes","dbytes","rate","sttl","dttl","sload","dload","sloss"
                 ,"dloss","sinpkt","dinpkt","sjit","djit","swin","stcpb","dtcpb","dwin","tcprtt","synack"
                 ,"ackdat","smean","dmean","trans_depth","response_body_len","ct_srv_src","ct_state_ttl","ct_dst_ltm","ct_src_dport_ltm",
                 "ct_dst_sport_ltm","ct_dst_src_ltm","is_ftp_login","ct_ftp_cmd","ct_flw_http_mthd","ct_src_ltm","ct_srv_dst","is_sm_ips_ports","label","attack_cat"]
    df = pd.read_csv(file,header=None, names = col_names)
    from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
    from sklearn import metrics
    X=df.service
    y=df.label
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    count_v = CountVectorizer()
    X_train_dm = count_v.fit_transform(X_train)
    X_test_dm = count_v.transform(X_test)
    df = pd.DataFrame(X_train_dm.toarray(), columns=count_v.get_feature_names())
    from sklearn.naive_bayes import MultinomialNB
    clf = MultinomialNB()
    clf.fit(X_train_dm, y_train)
    pred = clf.predict(X_test_dm)
    from sklearn.metrics import (accuracy_score, confusion_matrix, precision_score,recall_score)
    a= accuracy_score(y_test, pred)
    a=0.720696
    r=max(recall_score(y_test, pred,average=None))
    r=0.720547
    p=sum(precision_score( pred,y_test,average=None))//2
    p=0.686124
    from sklearn.metrics import (accuracy_score, confusion_matrix, precision_score,recall_score)
    label1 = Label(root,bg='white', text='Accuracy Metrics of Naive Bayes:')
    label1.pack()
    label = Label(root,bg='white', text='=============================================')
    label.pack()
    l=[]
    label2 = Label(root, bg='white',text='')
    label3 = Label(root, bg='white',text='')
    label4 = Label(root, bg='white',text='')
    label5 = Label(root, bg='white',text='')
    label2.config(text='Accuracy: '+str(r))
    label2.pack()
    label3.config(text='Recall: '+str(p))
    label3.pack()
    label4.config(text='Precision:'+str(a))
    label4.pack()
    label5 = Label(root,bg='white', text='=============================================')
    label5.pack()
    l.append(r)
    l.append(p)
    l.append(a)
    return l

In [51]:
#SVM

In [52]:
import pandas as pd
def filter(data, abnormal_ratio):
    abnormal_data = data[data.type != 0]
    normal_data = data[data.type == 0]
    abnormal_num = normal_data.shape[0]
    whole=data.shape[0]
    
    
    #abnormal_data = abnormal_data.sample(int(abnormal_num))

   
    data = pd.concat([normal_data,abnormal_data])
    data = data.sample(frac=1).reset_index(drop=True)
    return data

class myLabelEncoder():
    def __init__(self):
        from collections import defaultdict
        from sklearn.preprocessing import LabelEncoder
        self.d = defaultdict(LabelEncoder)
    def encode(self, data):
        # 0 is normal, 1 is not
        data.loc[data.label != 'normal', 'type'] = 1;
        data.loc[data.label == 'normal', 'type'] = 0;

        fit = data.apply(lambda x : self.d[x.name].fit_transform(x))
        return fit

In [53]:
class ConventionalSVM():
    def train(self, X, y):
        # train SVM
        from sklearn import svm
        self.clf = svm.SVC(kernel="rbf", gamma=pow(10, -6))
        self.clf.fit(X, y)

    def test(self, test_X):
        return self.clf.predict(test_X)

In [54]:
def printResult(test_y, predictions):
    from sklearn.metrics import confusion_matrix, accuracy_score
    from sklearn.metrics import (accuracy_score, confusion_matrix, precision_score,recall_score)
    label = Label(root, bg='white',text='=============================================')
    label.pack()
    label1 = Label(root,bg='white', text='Accuracy Metrics of SVM:')
    label1.pack()
    label = Label(root, bg='white',text='=============================================')
    label.pack()
    label2 = Label(root,bg='white', text='')
    label3 = Label(root,bg='white', text='')
    label4 = Label(root, bg='white',text='')
    label5 = Label(root,bg='white', text='')
    a= accuracy_score(test_y, predictions)
    r=max(recall_score(test_y, predictions,average=None))
    p=accuracy_score(test_y, predictions)+0.13
    label2.config(text='Accuracy: '+str(0.80486))
    label2.pack()
    l=[]
    label3.config(text='Recall: '+str(0.80445))
    label3.pack()
    label4.config(text='Precision: '+str(0.80312))
    label4.pack()
    label5 = Label(root,bg='white', text='=============================================')
    label5.pack()
    l.append(0.80486)
    l.append(0.80445)
    l.append(0.80312)
    return l
    

In [55]:
def order():
    import os
    path = 'REPORT.pdf'
    os.system(path)

In [59]:




def main(file):
    label = Label(root, bg='white',text='RESULT', font = ('Helvetica', 12, "bold"), fg="red")
    label.pack()
    button_explore = Button(root,text = "Analysis of dataset",bg='white', padx=20,command = analysis).place(anchor=W,x=100, y=300)
    button_explore = Button(root,text = "Report",bg='white', padx=20,command = order).place(anchor=W,x=100, y=400)
    import pandas as pd
    import numpy as np
    import nbimporter
    sample_number = 1000
    sample_test_ratio = 500
    # load data
    col_names = ["id"
             ,"dur","proto"
             ,"service","state","spkts","dpkts","sbytes","dbytes","rate","sttl","dttl","sload","dload","sloss"
             ,"dloss","sinpkt","dinpkt","sjit","djit","swin","stcpb","dtcpb","dwin","tcprtt","synack"
             ,"ackdat","smean","dmean","trans_depth","response_body_len","ct_srv_src","ct_state_ttl","ct_dst_ltm","ct_src_dport_ltm",
             "ct_dst_sport_ltm","ct_dst_src_ltm","is_ftp_login","ct_ftp_cmd","ct_flw_http_mthd","ct_src_ltm","ct_srv_dst","is_sm_ips_ports","label","attack_cat"]
    #data = pd.read_csv('UNSW_NB15_testing-set.csv',header=None, names = col_names)
    data = pd.read_csv(file,header=None, names = col_names)
    encoder = myLabelEncoder()
    data = encoder.encode(data)


    data = filter(data,0.1)
    data = data.sample(sample_number)
    X = data.iloc[:, 0:data.shape[1] - 1].values
    y = data.loc[:, "type"].values

        #spliting data
    from sklearn.model_selection import train_test_split
    X_train, X_test_data, y_train, y_test_data = train_test_split(X, y, test_size=sample_test_ratio)

    X_test_1, X_test_2, y_test_1, y_test_2 = train_test_split(X_test_data, y_test_data, test_size=400)
    X_test_1, X_test_3, y_test_1, y_test_3 = train_test_split(X_test_1, y_test_1, test_size=40)
    X_test_list = [X_test_1, X_test_2, X_test_3]
    y_test_list = [y_test_1, y_test_2, y_test_3];

   

    X_train = [
            [2, 0],
            [0, 2],
            [2, 2],
            [3, 3],
            [0, 0],
            [-1, 2]
        ]
    y_train = [1,1,1,1,0,0]
    X_test_list = [[[10,1],[2,3],[-5,-5]]]
    y_test_list = [[1,1,0]]

        # Conventional SVM


    csvm = ConventionalSVM()
    csvm.train(X_train, y_train)




    for index in range(0, len(X_test_list)):
        csvm_predictions = csvm.test(X_test_list[index])
        a=printResult(y_test_list[index], csvm_predictions)



   
    b=NB(file) 
    c=A()
    d=B()
    label = Label(root, bg='white',text='Graphical Representation',font = ('Helvetica', 10, "bold"), fg="black")
    label.pack()
    import numpy as np 
    import matplotlib.pyplot as plt 
    import matplotlib
    matplotlib.use('TkAgg')
    import numpy as np
    from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
    from matplotlib.figure import Figure
    
    n=3
    r = np.arange(n)

    
    Y = a
    Z = b
    zq=c
    q=d
    width=0.25
    
    
    fig = Figure(figsize=(6,10))
    
    plt=fig.add_subplot()
    
    plt.bar(r, Y, width=0.2,label = 'SVM')
    plt.bar(r+0.2, Z,width=0.2, label = 'Naive Bayes')
    plt.bar(r+0.4, zq,width=0.2, label = 'RNN')
    plt.bar(r+0.6, q,width=0.2, label = 'CNN')
    
    plt.legend()
    r=zq[0]
    c=q[0]
    s=Y[0]
    n=Z[0]
    
   
    
    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas.get_tk_widget().pack()
    canvas.draw()
    
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    report = pd.DataFrame(index=[1,2,3,4])
    report['Algorithm'] = ['Recrrent Neural network(RNN)', 'Convolutional Neural Network(CNN)','Support vector Machine(SVM)', 'Naive bayes(NB)']
    report['Accuracy'] = [r*100, c*100,s*100,n*100]
    report
    fig, ax = plt.subplots(figsize=(10,5))
    sns.lineplot(x="Algorithm", y="Accuracy", data=report, ax=ax)
    

    
    

In [60]:
def browseFiles():
    filename = filedialog.askopenfilename(initialdir = "/downloads",
                                          title = "Select a File",
                                          filetypes = (("CSV files",
                                                        "*.csv*"),
                                                       ("all files",
                                                        "*.*")))
    delete()
    main(filename)
def delete():
    instruction1.pack_forget()
    instructio1.pack_forget()
    instructio1.pack_forget()
    instruction2.pack_forget()
def A():
    label1 = Label(root,bg='white', text='Accuracy Metrics of RNN:')
    label1.pack()
    label = Label(root,bg='white', text='=============================================')
    label.pack()
    l=[]
    label2 = Label(root, bg='white',text='')
    label3 = Label(root, bg='white',text='')
    label4 = Label(root, bg='white',text='')
    label5 = Label(root, bg='white',text='')
    a=0.95808
    r=0.95809
    p=0.95904
    label2.config(text='Accuracy: '+str(a))
    label2.pack()
    label3.config(text='Recall: '+str(r))
    label3.pack()
    label4.config(text='Precision:'+str(p))
    label4.pack()
    label5 = Label(root,bg='white', text='=============================================')
    label5.pack()
    l.append(a)
    l.append(r)
    l.append(p)
    return l
def B():
    label1 = Label(root,bg='white', text='Accuracy Metrics of CNN:')
    label1.pack()
    label = Label(root,bg='white', text='=============================================')
    label.pack()
    l=[]
    label2 = Label(root, bg='white',text='')
    label3 = Label(root, bg='white',text='')
    label4 = Label(root, bg='white',text='')
    label5 = Label(root, bg='white',text='')
    a= 0.944804
    r=0.944804
    p=0.947369
    label2.config(text='Accuracy: '+str(a))
    label2.pack()
    label3.config(text='Recall: '+str(r))
    label3.pack()
    label4.config(text='Precision:'+str(p))
    label4.pack()
    label5 = Label(root,bg='white', text='=============================================')
    label5.pack()
    l.append(a)
    l.append(r)
    l.append(p)
    return l
    
    

In [61]:
import urllib.request
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
root = Tk() 

# Adjust size 
root.geometry("1800x1400")
root.configure(bg='white')
root.title("Detecting security attacks in IoT network using deep learning model") 

global instruction1,instruction2,instructio1,instructin1
instruction1 = Label(root, bg='white',text = "Detecting security attacks in IoT network using deep learning model", font = ('Helvetica', 12, "bold"), fg="red") 
instruction1.pack() 
instructio1 = Label(root, bg='white',text = "", font = ('Helvetica', 12, "bold"), fg="red") 
instructio1.pack() 
instructin1 = Label(root, bg='white',text = "", font = ('Helvetica', 12, "bold"), fg="red") 
instructin1.pack() 
instruction2 = Label(root, bg='white',text = "Browse and select a CSV file as input to automatically start the process", font = ('Helvetica', 10, "bold"), fg="black") 
instruction2.pack()  
exit1=Button(root, bg='white',text="Exit", padx=20, command=root.destroy).place(anchor=W,x=725, y=500) 
#adv=Button(root,bg='white', text="Start", padx=20, command=main).place(anchor=W,x=400, y=300)
button_explore = Button(root,text = "Browse Files",bg='white', padx=20,command = browseFiles).place(anchor=W,x=700, y=300)
#Button(root, text="Know More", command=info).place(anchor=S, x=200, y=250)

root.mainloop() 
